In [47]:
def Strategy_3():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    stocks = client.Common.Stock.find({})
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = int(p5 / 5)
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10),
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    if len(found) > 0:
        print pandas.DataFrame(found[:10])
    else:
        print("No Result")

In [50]:
Strategy_3()

     code    factor   name  price_5  price_today  volume_10  volume_2
0  002931  4.264010   锋龙股份       35        50.80       8155     34773
1  601518  3.603308   吉林高速        3         3.95     108147    389687
2  600853  3.513564   龙建股份        4         5.10      82021    288186
3  000755  3.183396  *ST三维        5         5.27      27187     86547
4  600370  3.114625    三房巷        2         3.05      57867    180234
5  002691  3.043113   冀凯股份       23        25.40      14149     43057
6  600738  3.006270   兰州民百        7         7.63      20575     61854
7  300537  3.005118   广信材料       17        19.27      45522    136799
8  600695  2.777235   绿庭投资        8         8.02      26207     72783
9  002627  2.718123   宜昌交运       18        19.97      14556     39565
